## Gnowee

This notebook contains several examples of how to run the Gnowee for different problem and input types.  Additionally, a batch specification is included that can be used to develop statistics to illustrate Gnowee's performance under various conditions.

### Example 1 - Engineering Benchmarks
This scipt is the bare minimum needed to run Gnowee and display the results of the optimization. 

It requires the use of one of the built-in constrained benchmarks.  All program deafults are left intact.

In [1]:
# Gnowee Modules
import Gnowee
from ObjectiveFunction import ObjectiveFunction
from GnoweeUtilities import ProblemParameters
from GnoweeHeuristics import GnoweeHeuristics

# Select optimization problem type and associated parameters
gh = GnoweeHeuristics()
gh.set_preset_params('spring')
print gh

# Run optimization
(timeline) = Gnowee.main(gh)
print '\nThe result:\n', timeline[-1]

  GnoweeHeuristics:
Population = 25
Sampling Method = lhc
Discovery Fraction = 0.2
Elitism Fraction = 0.2
Levy Fraction = 1.0
Levy Alpha = 0.5
Levy Gamma = 1
Levy Independent Samples = 1
Levy Scaling Parameter = 10.0
Constraint Violaition Penalty = 0.0
Max # of Generations = 20000
Max # of Function Evaluations = 200000
Convergence Tolerance = 1e-06
Stall Limit = 10000
Optimal Convergence Tolerance = 0.01
     Attributes Inhereted from ProblemParameters:
  ProblemParameters:
Lower Bounds: [ 0.05  0.25  2.  ]
Upper Bounds: [  2.    1.3  15. ]
Variable Types: ['c', 'c', 'c']
Continuous ID Vector: [1 1 1]
Integer ID Vector: [0 0 0]
Discrete ID Vector: [0 0 0]
Combinatorial ID Vector: [0 0 0]
Discrete Values: [] 
Global Optimum: 0.012665
Plot Title: \textbf{Spring Optimization using Gnowee}
Histogram Title: \textbf{Function Evaluations for Spring Optimization using Gnowee}
Variable Names: ['\\textbf{Fitness}', '\\textbf{Width}', '\\textbf{Diameter}', '\\textbf{Length}']
  ObjectiveFunction:

### Example 2 - Continuous, n-Dimensional Function Benchmarks
This shows how to run one of the n-Dimensional Benchmarks included in the package.  

It also illustrates how to change a couple of the preset Gnowee algorithm settings.  The settings were optimized across a wide range of problem types, but they are not optimal for all classes of problems.  Refer to the <a href='../Benchmarks/plots/HyperOptimization'>hyper-optimization plots</a> or the <a href='../Benchmarks/plots/HyperOptimization'>hyper-optimization ipython notebooks</a> for more information and results.

In [ ]:
# Gnowee Modules
import Gnowee
from ObjectiveFunction import ObjectiveFunction
from GnoweeUtilities import ProblemParameters
from GnoweeHeuristics import GnoweeHeuristics

# Select optimization problem type and associated parameters
gh = GnoweeHeuristics()
gh.set_preset_params('rastrigin', dimension=3)

# Change any desired program settings
gh.initSampling = 'nolh'
gh.population = 50
print gh

# Run optimization
(timeline) = Gnowee.main(gh)
print '\nThe result:\n', timeline[-1]

### Example 3 - User Specified Function
This example shows how to run a user specified function.  This requires the user to specify the objective function, constraints (if any), bounds for each variable, and the type of each variable.  Additional problem parameters can be specified as appropriate.  Refer to the <a href='GnoweeUtilities.py'>ProblemParameters</a> class for further information.

This example utilizes the <a href='ExampleFunction.py'>ExampleFunction</a> module included with the Gnowee package as an example for how to import user specified objective functions.  

A similar process can be followed for constraints, but a built-in constraint is used in this example.

In [ ]:
# Gnowee Modules
import Gnowee
from ObjectiveFunction import ObjectiveFunction
from Constraints import Constraint
from GnoweeHeuristics import GnoweeHeuristics

# User Function Module
from ExampleFunction import spring

# Select optimization problem type and associated parameters
gh = GnoweeHeuristics(objective=ObjectiveFunction(spring), constraints=Constraint('spring', 0.0),
                      lowerBounds=[0.05, 0.25, 2.0], upperBounds=[2.0, 1.3, 15.0],
                      varType=['c', 'c', 'c'], optimum=0.012665)
print gh

# Run optimization
(timeline) = Gnowee.main(gh)
print '\nThe result:\n', timeline[-1]

### Example 4 - TSP Benchmarks
This example shows how to use the built-in TSP benchmarks using the standard combinatorial heuristics in Gnowee.  These runs will not incorporate distance based modifiers of the heuristics used to compare algorithm performance as shown in the <a href='..\Benchmarks\results\Gnowee_Benchmark_Results.pdf'>benchmark results</a>.

In [ ]:
import os

# Gnowee Modules
import Gnowee
from ObjectiveFunction import ObjectiveFunction
from GnoweeUtilities import ProblemParameters
from GnoweeHeuristics import GnoweeHeuristics
from TSP import TSP

# Select optimization problem type and associated parameters
gh = GnoweeHeuristics()
tspProb = TSP()
tspProb.read_tsp(os.path.pardir+'/Benchmarks/TSPLIB/eil51.tsp')
tspProb.build_prob_params(gh)

# Modify convergence parameters
gh.maxGens = 750
gh.stallLimit = 15000
gh.optConTol = 0.1
print gh

# Run optimization
(timeline) = Gnowee.main(gh)
print '\nThe result:\n', timeline[-1]

In [ ]:
from ObjectiveFunction import ObjectiveFunction
from Constraints import Constraint

optFunc = ObjectiveFunction(method='mi_pressure_vessel')
print optFunc.func([38.77342202, 223.7237953, 0.75, 0.375])

conFunc = Constraint(method='mi_pressure_vessel', constraint=0.0)
print conFunc.func([38.77342202, 223.7237953, 0.75, 0.375])

## Batch Gnowee

This scipt runs multipe instances of Gnowee against an optimization prolem.  It is useful for benchmarking and understanding the distribution of solutions obtained by Gnowee for a particular problem.  Statistics and plots are obtained and shown following the optimization process.  

In [ ]:
import os
import sys
import numpy as np
import math as m

from operator import attrgetter

# Gnowee Modules
import Gnowee
import OptiPlot as op
from ObjectiveFunction import ObjectiveFunction
from GnoweeHeuristics import GnoweeHeuristics
from GnoweeUtilities import Event, ProblemParameters
from TSP import TSP

# Initialize variables
gh = GnoweeHeuristics()
maxIter = 50     # Number of algorithm iterations
evalInterval = 500  # Fuction eval interval at which the fitness is sampled.  
history = []        # List that contains the final timeline results from each optimization run 

# History of fitness vs function evals [Feval, Fit avg, Fit std, counter]
fevalHistory = np.array([[i*evalInterval,0.0,0.0,0] for i in \
                            range(int(gh.maxFevals/evalInterval)+1)])

# Select optimization problem type and associated parameters
#problem = 'welded_beam'
#problem = 'pressure_vessel'
#problem = 'speed_reducer'
problem = 'spring'
#problem = 'mi_spring'
#problem = 'mi_pressure_vessel'
#problem = 'mi_chemical_process'

#problem = 'ackley'
#problem = 'shifted_ackley'
#problem = 'dejong'
#problem = 'shifted_dejong'
#problem = 'easom'
#problem = 'shifted_easom'
#problem = 'griewank'
#problem = 'shifted_griewank'
#problem = 'rastrigin'
#problem = 'shifted_rastrigin'
#problem = 'rosenbrock'
#problem = 'shifted_rosenbrock'

#problem = 'tsp'
#tsplib = 'eil51'
#tsplib = 'st70'
#tsplib = 'pr107'
#tsplib = 'bier127'
#tsplib = 'ch150'

# Run optimization
for i in range(0,maxIter,1):
    print "Run #{}".format(i)

    # Set optimization settings
    if problem != 'tsp':
        gh.set_preset_params(problem, 'Gnowee', dimension=3)
        gh.stallLimit = 10000
    else:
        tspProb = TSP()
        tspProb.read_tsp(os.path.pardir+'/Benchmarks/TSPLIB/{}.tsp'.format(tsplib))
        tspProb.build_prob_params(gh)
        gh.stallLimit = 15000
    #gh.maxFevals = 10000
    #gh.optConvTol = 1E-16
    (timeline) = Gnowee.main(gh)
    
    # Save final timeline data for future processing
    minGen = min(timeline, key=attrgetter('fitness'))
    history.append(Event(minGen.generation,minGen.evaluations,minGen.fitness,
                             minGen.design))

    # Update Fitness vs Feval History
    k = 1 
    
    # Compute the average and standard deviation using a recurrence relation
    for j in range(0, len(fevalHistory),1):
        while timeline[k].evaluations < fevalHistory[j,0] and k+1 < len(timeline):  
            k += 1
        if k+1 == len(timeline) and timeline[k].evaluations < fevalHistory[j, 0]:
            # Initialize the array on the first run
            if fevalHistory[j, 3] == 0:
                fevalHistory[j, 1] = timeline[k].fitness
                fevalHistory[j, 2] = 0.0
                fevalHistory[j, 3] += 1
            else:
                oldMean = fevalHistory[j,1]
                fevalHistory[j, 3] += 1
                fevalHistory[j, 1]=fevalHistory[j, 1]\
                                 +(timeline[k].fitness-fevalHistory[j,1])/fevalHistory[j,3]
                fevalHistory[j, 2] = fevalHistory[j, 2] \
                                 +(timeline[k].fitness-oldMean)*(timeline[k].fitness \
                                                                 -fevalHistory[j,1])                
            break
        else:
            # Initialize the array on the first run
            if fevalHistory[j, 3] == 0:
                fevalHistory[j, 1] = timeline[k-1].fitness
                fevalHistory[j, 2] = 0.0
                fevalHistory[j, 3] += 1
            else:    
                oldMean = fevalHistory[j,1]
                fevalHistory[j, 3] += 1
                fevalHistory[j, 1] = fevalHistory[j,1]+(timeline[k-1].fitness \
                                     -fevalHistory[j,1])/fevalHistory[j,3]
                fevalHistory[j, 2] = fevalHistory[j,2]+(timeline[k-1].fitness \
                                     -oldMean)*(timeline[k-1].fitness-fevalHistory[j,1])
            
    #op.Plot_Feval_Hist(data=fevalHistory)
    
# Calculate averages and standard deviations
tmp = []
for i in range(len(history[-1].design)):
    if gh.objective.func.__name__ != "tsp":
        tmp.append(sum(c.design[i] for c in history)/float(len(history)))
averages = Event(sum(c.generation for c in history)/float(len(history)),
                     sum(c.evaluations for c in history)/float(len(history)),
                     sum(c.fitness for c in history)/float(len(history)),tmp)

tmp=[]
if maxIter > 1:
    for i in range(len(history[-1].design)):
        if gh.objective.func.__name__ != "tsp":
            tmp.append(m.sqrt(sum([(c.design[i]-averages.design[i])**2 for c in history]) \
                              /(len(history) - 1)))
    stdDev = Event(m.sqrt(sum([(c.generation-averages.generation)**2 for c in history]) \
                               /(len(history) - 1)),
                        m.sqrt(sum([(c.evaluations- averages.evaluations)**2 for c in history]) \
                               /(len(history) - 1)),
                        m.sqrt(sum([(c.fitness - averages.fitness)**2 for c in history]) \
                               /(len(history) - 1)),tmp)
else:
    tmp = np.zeros(len(history[-1].design))
    stdDev = Event(0, 0, 0.0, tmp)

# Trim empty feval histories 
if fevalHistory[-1,3 ] == 0:
    fevalHistory = np.array([fevalHistory[tmp, :] for tmp in \
                            range(len(fevalHistory[0:(np.argmin(fevalHistory[:, 3]))]))])

# Compute relative error (%Diff) for feval histories
if gh.optimum == 0.0:
    fevalHistory[:, 1] = fevalHistory[:, 1]*100
else:
    fevalHistory[:, 1] = (fevalHistory[:, 1]-gh.optimum)/gh.optimum*100
    fevalHistory[:, 1] = [cur if i >= 0 else 0 for cur in fevalHistory[:, 1]]

# Compute standard deviation for feval histories
for i in range(0, len(fevalHistory)):
    if fevalHistory[i, 2] != 0.0:
        fevalHistory[i, 2] = np.sqrt(fevalHistory[i, 2]/(fevalHistory[i, 3]-1))*100

# Output average results to the user
print "\nThe Average Optimized Solution for {}:".format(gh.objective.func.__name__)
print "================================"
print "Design:"
for i in range(len(averages.design)):
    print "   var %d: %4.6f $\mypm$ %4.5f " %(i+1, averages.design[i], stdDev.design[i])
print "Fitness: %4.6f $\mypm$ %4.5f " %(averages.fitness, stdDev.fitness)
print "Funct Evals: %d $\mypm$ %d " %(averages.evaluations, stdDev.evaluations)
print "Generations: %3.1f $\mypm$ %3.1f "  %(averages.generation, stdDev.generation)
if gh.optimum == 0.0:
    print "The performance metric is %4.1f" %(averages.fitness*(averages.evaluations \
                                                                +3*stdDev.evaluations))
else:
    print "The performance metric is %4.1f" %(abs((averages.fitness-gh.optimum) \
                                                  /gh.optimum) \
                                              *(averages.evaluations+3*stdDev.evaluations))

#Determine the best values obtained
best = min(history, key=attrgetter('fitness'))

# Output best result to the user
print "\nThe Best Optimized Solution for {}:".format(gh.objective.func.__name__)
print "================================"
print "Design:"
for i in range(len(averages.design)):
    print "   var %d: %4.6f " %(i+1,best.design[i])
print "Fitness: %4.6f " %best.fitness
print "Funct Evals: %d " %best.evaluations
print "Generations: %d "  %best.generation
if gh.optimum == 0.0:
    print "The performance metric is %4.1f" %(best.fitness*best.evaluations)
else:
    print "The performance metric is %4.1f" %(abs((best.fitness-gh.optimum) \
                                                  /gh.optimum)*best.evaluations)

#Plot the optimization process
if  gh.objective.func.__name__ != "tsp":
    op.plot_vars(timeline, lowBounds=gh.lb, upBounds=gh.ub, title=gh.pltTitle, label=gh.varNames)
fevals=[tmp.evaluations for tmp in history]
if maxIter >1:
    op.plot_hist(fevals,title=gh.histTitle)

    op.plot_feval_hist(data=fevalHistory)

In [ ]:
# Print the function eval history
s = 'np.array(['
for i in range(0,len(fevalHistory)-1,1):
    s += '[{}, {}, {}, {}],'.format(fevalHistory[i,0],fevalHistory[i,1],fevalHistory[i,2],fevalHistory[i,3])
s += '[{}, {}, {}, {}]])'.format(fevalHistory[-1,0],fevalHistory[-1,1],fevalHistory[-1,2],fevalHistory[-1,3])
print s